In [1]:
import logging
from copy import deepcopy
import pickle

import numpy as np
import pandas as pd
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import MultilayerPerceptronClassifier
from regression import CustomRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# conf = SparkConf()
# spark = SparkContext.getOrCreate(conf=conf)

In [2]:
train = pd.read_pickle('train_df.p')
test = pd.read_pickle('test_df.p')

# keys = set(train.keys())
# keys.remove('000001.SS')
# keys.remove('Target')
# ass = VectorAssembler(inputCols=list(keys), outputCol="features")
# df = spark.createDataFrame(train)
# output = ass.transform(df)
# test_df = spark.createDataFrame(test)
# test_fe = ass.transform(test_df)

In [9]:
lr = LinearRegression(featuresCol="features", labelCol='Target', maxIter=10000)
model = lr.fit(output)
test_re = model.transform(test_fe)

In [6]:
algorithm = CustomRegression(MLPRegressor(hidden_layer_sizes=(16,)))
model = algorithm.fit(np.array(output.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect()), 
                      output.select('Target').rdd.map(lambda x: list(x)[0]))

In [7]:
# data_df = test_fe.select('features').toPandas()
model.predict(test_fe.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect()).collect()

Help on class CustomRegression in module regression.algorithms:

class CustomRegression(MassRegressionAlgorithm)
 |  Custom mass univariate regression algorithm.
 |  
 |  Parameters
 |  ----------
 |  
 |  algorithm : object
 |      An object encapsulating a regression algorithm.
 |  
 |      Can be a scikit-learn algorithm or a custom algorithm that adheres to a
 |      scikit-learn style API. Must implement a fit(X, y) method that takes a
 |      deisgn matrix and response vector (X and y, both NumPy ndarrays) and
 |      returns a fitted model. The fitted model should expose data members
 |      coef_ and intercept_ that contain the fitted coefficients and intercept
 |      (if any). The fitted model should also implement a score(X, y) method
 |      giving the r-squared of fit for the given data and a predict(X) method
 |      that predicts responses given a design matrix.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, algorithm)
 |  
 |  -----------------------------------

In [18]:
alg = MLPRegressor(hidden_layer_sizes=(16,))


In [30]:
from sklearn.datasets import make_regression
X, Y = make_regression(n_samples=100, n_features=3, n_informative=3, n_targets=1, noise=1.0)
algorithm = CustomRegression(MLPRegressor(hidden_layer_sizes=(16,)))
model = algorithm.fit(X, Y.T)

In [20]:
Y = output.select('Target').rdd.map(lambda x: list(x)[0])
X = np.array(output.select('features').rdd.map(lambda x: np.array(list(x)[0])).collect())
from regression.utils import toseries
y = toseries(Y)

In [ ]:
output.select('Target').rdd.map(lambda x: list(x)[0]).collect()

In [34]:
toseries(Y).map(lambda v: deepcopy(alg).fit(X, v)).first()

array([ MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(16,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)], dtype=object)

In [2]:
df = spark.read.load('/Users/warn/PycharmProjects/Dissertation/src/stockforecaster/test.parquet')

In [13]:
input_num = 26
layers = [input_num, input_num / 3 * 2, input_num / 3, 2]
mlpc = MultilayerPerceptronClassifier(featuresCol="features", labelCol='ChangeDirection',
                                      layers=layers,
                                      predictionCol='DirPrediction')
model = mlpc.fit(df)

In [15]:
# pdf = df.toPandas()
# pdf['features'].values
spark.stop()

In [4]:
# import numpy as np
pnparray = pdf['features'].values
container = np.zeros((pnparray.shape[0], len(pnparray[0])))
for i in range(pnparray.shape[0]):
    container[i, :] = pnparray[i][:]

In [2]:
with open('/Users/warn/PycharmProjects/Dissertation/src/stockforecaster/ann_result.p') as f:
    data = pickle.load(f)
    
df = spark.read.load('/Users/warn/PycharmProjects/Dissertation/src/stockforecaster/test_df.parquet')

In [3]:
pdf = pd.DataFrame(data, columns=['prediction'])
sdf = spark.createDataFrame(pdf)
# df.withColumn('prediction', getattr(sdf, 'prediction'))

In [4]:
lr = LinearRegression()
print lr.explainParams()

elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: the solver algorithm for optimization. If this is not set or empty, default value is 'auto'. (default: auto)
standardization: whether to standardize the training features before fitting the model. (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)


In [6]:
print train.keys()


Index([u'Open', u'High', u'Low', u'Close', u'Volume', u'MACD_12_26_9',
       u'MACD_7_14_9', u'SMA_3', u'SMA_13', u'SMA_21', u'EMA_5', u'EMA_13',
       u'EMA_21', u'ROC_13', u'ROC_21', u'RSI_9', u'RSI_14', u'RSI_21',
       u'Hang Seng Index', u'000001.SS', u'USD2HKD', u'EUR2HKD', u'12 Months',
       u'6 Months', u'Overnight', u'gold price in CNY', u'Target'],
      dtype='object')


,Open,High,Low,Close,Volume,MACD_12_26_9,MACD_7_14_9,SMA_3,SMA_13,SMA_21,...,RSI_21,Hang Seng Index,000001.SS,USD2HKD,EUR2HKD,12 Months,6 Months,Overnight,gold price in CNY,Target
Date,,,,,,,,,,,,,,,,,,,,,
2010-08-10,-0.525133,-0.536976,-0.531301,-0.532645,-0.479639,0.517976,0.601948,-0.565265,-0.779584,-0.837765,...,0.600936,-0.143188,-0.598761,-0.507745,-0.109772,0.425270,0.275865,-0.949112,-0.494857,72.7583
2010-08-11,-0.570052,-0.532690,-0.518161,-0.519694,-0.564394,0.552253,0.616052,-0.554771,-0.762768,-0.826804,...,0.636368,-0.172561,-0.591156,-0.490534,-0.169305,0.296537,0.155176,-0.949112,-0.460904,73.5445
2010-08-12,-0.580749,-0.526257,-0.542253,-0.495948,-0.235184,0.589909,0.639254,-0.551773,-0.746140,-0.814571,...,0.698675,-0.203535,-0.611067,-0.376936,-0.227407,0.347129,0.155176,-0.949112,-0.437542,72.6154
2010-08-13,-0.546522,-0.526257,-0.520352,-0.524010,-0.516255,0.595156,0.598590,-0.548775,-0.731401,-0.802641,...,0.525005,-0.209135,-0.591654,-0.242685,-0.218115,0.485067,0.206891,-0.949112,-0.429633,71.0430
2010-08-16,-0.604279,-0.605573,-0.579486,-0.571501,-0.477135,0.561963,0.491888,-0.566764,-0.720442,-0.792528,...,0.267040,-0.202484,-0.557447,-0.197935,-0.146125,0.365474,0.099116,-0.949112,-0.401100,71.6148
2010-08-17,-0.619249,-0.573418,-0.570725,-0.554231,-0.692802,0.542319,0.432079,-0.587001,-0.708537,-0.780598,...,0.323438,-0.198332,-0.551117,-0.239243,-0.222710,0.296537,0.038771,-0.949112,-0.401688,71.9006
2010-08-18,-0.548662,-0.560557,-0.542253,-0.545599,-0.630982,0.527742,0.393240,-0.594497,-0.695594,-0.769334,...,0.351331,-0.217147,-0.554593,-0.352840,-0.218421,0.232170,-0.025858,-0.949112,-0.421997,72.1151
2010-08-19,-0.561496,-0.564843,-0.542253,-0.539121,-0.610374,0.515861,0.366455,-0.583254,-0.685768,-0.757767,...,0.372544,-0.208989,-0.541112,-0.301205,-0.184315,0.108007,-0.142262,-0.949112,-0.383685,71.2574
2010-08-20,-0.593582,-0.596998,-0.572916,-0.565026,-0.803390,0.484580,0.305266,-0.587002,-0.680005,-0.748320,...,0.232727,-0.223858,-0.569511,-0.108434,-0.273767,-0.025296,-0.267236,-0.983369,-0.408839,71.0072


In [7]:
tran = MinMaxScaler(feature_range=(-1, 1))
traned = tran.fit_transform(train)

ValueError: can not merge DataFrame with instance of type <type 'numpy.ndarray'>

array([-0.53697644, -0.53269049, -0.52625707, ...,  0.69095572,
        0.74194322,  0.73894396])